In [1]:
import pandas as pd
import re

#!python --version  #Python 3.8.2
#pd.__version__     #1.0.3 
#re.__version__     #2.2.1

In [2]:
#CUSTOM SETTINGS set these as appropriate for your environment

# Enter the path to the local data files:
path_datafiles = "OriginalData/"

# Download the state names/abbreviations from US Postal Service Publication 28
#              https://pe.usps.com/text/pub28/28apb.htm
#        Required to verify that state abbreviations are valid.
#        Expects first column is state name, second column is state abbreviation.
# Enter the path to the file on your system.
postal_file = '20200712_StateAbbreviations.txt'

# Get quarterly LESO Shipment and Cancellation data file from 
#     Defense Logicstics Agency Law Enforcement Support Office Public Information
# Orginal name of the data file should be in the form:
#      DISP_Shipments_Cancellations_mmddyyyy_mmddyyyy.xlsx  
# Enter the local file name
LESO_file = "DISP_Shipments_Cancellations_01012020_03312020.xlsx"

DISP_Shipments_Cancellations_mmddyyyy_mmddyyyy.xlsx is downloaded from:    
https://www.dla.mil/DispositionServices/Offers/Reutilization/LawEnforcement/PublicInformation/   
The following is an image of the relevant section of the website:

![test](Images/DISP_Shipments_CancellationsXLSX.png)

## Check DISP_Shipments_Cancellations_mmddyyyy_mmddyyyy.xlsx

This notebook checks that the schema matches previous versions of the file. It checks for null/NaN data, some unique values, and that 'State' is a valid two-letter abbreviation. It expects that the XLSX file has two sheets labeled 'SHIPMENTS' and 'CANCELLATIONS' The two sheets have the different columns.

###### Expected Columns in 'SHIPMENTS' sheet:

__State__: two digit postal abbreviation<br>
> TYPE:str LENGTH: 2 CHARACTER_SET: [A-Z]   

__Station Name (LEA)__: descriptive name of agency requesting equipment<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__Requisition ID__: ???unique??? alphanumeric string<br>
> TYPE:str LENGTH: 14 CHARACTER_SET: [A-z0-9]   

__FSC__: Federal Supply Classification Group number<br>
part of NATO Stock Number, see https://en.wikipedia.org/wiki/NATO_Stock_Number<br>
> TYPE:str LENGTH:4 CHARACTER_SET: [0-9] varies (xx: FSG yy: FSC)   

__NIIN__: National Item Identification number<br>
part of NATO Stock Number, see https://en.wikipedia.org/wiki/NATO_Stock_Number<br>
> TYPE:str LENGTH:9 CHARACTER_SET: varies {xx:CC||NCB yyy-yyyy: non-standard item code}   

__Item Name__: descriptive item name<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__UI__: unit increment<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__Quantity__: number of items requested<br>
> TYPE:int LENGTH: varies CHARACTER_SET: [0-9]   

__Acquisition Value__: value of requested items in dollars<br>
> TYPE:float64 LENGTH: varies CHARACTER_SET: [0-9.]   

__Date Shipped__: ???date shipped???<br>
> TYPE:datetime64 LENGTH:29 CHARACTER_SET: yyyy-mm-ddT00:00:00.000000000   

__Justification__: descriptive text<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies

###### Expected Columns in 'CANCELLATIONS' sheet:

__Cancelled By__: ???cancelling agency???<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__RTD Ref__: ???unique??? reference number<br>
> TYPE:int LENGTH: 6 or 7 CHARACTER_SET: [0-9]  

__State__: two digit postal abbreviation<br>
> TYPE:str LENGTH: 2 CHARACTER_SET: [A-Z]   

__Station Name (LEA)__: descriptive name of agency requesting equipment<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__FSC__: Federal Supply Classification Group number<br>
part of NATO Stock Number, see https://en.wikipedia.org/wiki/NATO_Stock_Number<br>
> TYPE:str LENGTH:4 CHARACTER_SET: [0-9] {aabb: FSG(aa),FSC(bb)}   

__NIIN__: National Item Identification number<br>
part of NATO Stock Number, see https://en.wikipedia.org/wiki/NATO_Stock_Number<br>
> TYPE:str LENGTH:9 CHARACTER_SET: [A-Z0-9] {aabbbbbbb: CC||NCB(aa),non-standard item code(bbbbbbb)}   

__Item Name__: descriptive item name<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__UI__: unit increment<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__Quantity__: number of items requested<br>
> TYPE:int LENGTH: varies CHARACTER_SET: [0-9]   

__Acquisition Value__: value of requested items in dollars<br>
> TYPE:float64 LENGTH: varies CHARACTER_SET: [0-9.]   

__Date Requested__: ???date requested???<br>
> TYPE:datetime64 LENGTH:29 CHARACTER_SET: yyyy-mm-ddThh:mm:ss.000000000   

__Justification__: descriptive text<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__Reason Cancelled__: ???descriptive reason for cancellation???<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies

### Read Data From xlsx File:

In [3]:
ship_cancel_df = pd.read_excel("file:" + path_datafiles + LESO_file, sheet_name=None)
#transfer_df is a dictionary of two sheets in filename
#keys are 'SHIPMENTS', 'CANCELLATIONS'
#values are a single dataframe

### Expected Values

In [4]:
# based on sheets from previous files
expected_sheets = ['SHIPMENTS','CANCELLATIONS']

# based on columns from previous files
expected_shipments_columns = ['State', 'Station Name (LEA)', 'Requisition ID', 
                              'FSC', 'NIIN', 'Item Name', 'UI', 'Quantity', 
                              'Acquisition Value', 'Date Shipped', 'Justification']
expected_cancellations_columns = ['Cancelled By', 'RTD Ref', 'State', 'Station Name (LEA)', 
                                  'FSC', 'NIIN', 'Item Name', 'UI', 'Quantity', 
                                  'Acquisition Value', 'Date Requested', 'Justification', 
                                  'Reason Cancelled']

# dictionary based on U.S. Postal data from 'fullpath_postalfile'
#     key: state abbreviation, value: state name
expected_state_abbreviations = pd.read_csv(path_datafiles + postal_file,header=None,
                                           quotechar = "'").\
                                           set_index([1])[0].to_dict() 
#len(expected_state_abbreviations) #expect 59 U.S. states and territories

### Useful Functions for Checking the Values

In [5]:
def get_unique_values(df: pd.DataFrame,col: str) -> list:
    '''Returns a list of the unique values in a column given a dictionary of dataframes.    
    
    '''
    unique_values_list = []
    for dict_key in df:
        unique_values_array = df[dict_key][col].unique()
        for val in df[dict_key][col].unique():
            unique_values_list.append(val)
    return unique_values_list

def get_unexpected_values(to_check: set,expect: set)-> set:
    '''Returns a set of unexpected values, empty if none found.
    
    '''
    return to_check.difference(expect)

### Check for 2 Sheets

In [6]:
#type(ship_cancel_df) #dict
#len(ship_cancel_df.values()) #2
#for val in ship_cancel_df.values():
#    print('\n',type(val)) #<class 'pandas.core.frame.DataFrame'>
unexpected_sheets = get_unexpected_values(set(ship_cancel_df.keys()),
                                          set(expected_sheets))
if (len(ship_cancel_df.keys()) != 2) | (len(unexpected_sheets) != 0):
    print('XLSX has unexpected sheets:',ship_cancel_df.keys())
else:
    shipments_df = ship_cancel_df[expected_sheets[0]]
    cancellations_df = ship_cancel_df[expected_sheets[1]]

In [7]:
print('shipments_df has shape:',shipments_df.shape)
print('cancellations_df has shape:',cancellations_df.shape)

shipments_df has shape: (6353, 11)
cancellations_df has shape: (6640, 13)


In [8]:
print('shipments_df has these default data types:',shipments_df.dtypes)
print('cancellations_df has these default data types:',cancellations_df.dtypes)

shipments_df has these default data types: State                         object
Station Name (LEA)            object
Requisition ID                object
FSC                            int64
NIIN                          object
Item Name                     object
UI                            object
Quantity                       int64
Acquisition Value            float64
Date Shipped          datetime64[ns]
Justification                 object
dtype: object
cancellations_df has these default data types: Cancelled By                  object
RTD Ref                        int64
State                         object
Station Name (LEA)            object
FSC                            int64
NIIN                          object
Item Name                     object
UI                            object
Quantity                       int64
Acquisition Value            float64
Date Requested        datetime64[ns]
Justification                 object
Reason Cancelled              object
dtype: o

### Check the Shipments dataframe

###### QUESTION: Do the 'State' abbreviations in the 'SHIPMENTS' dataframe match US postal abbreviations?

In [9]:
incorrect_state_abbr = [state_abbr for state_abbr in shipments_df['State'] 
                         if state_abbr not in expected_state_abbreviations]
if len(incorrect_state_abbr) > 0:
    print('These states have inconsistant state/territory abbreviations:',incorrect_state_abbr)
else:
    print('No inconsistant state abbreviations were found.')

No inconsistant state abbreviations were found.


###### QUESTION: What columns are in the 'SHIPMENTS' dataframe?

In [10]:
new_columns = get_unexpected_values(set(shipments_df.columns), set(expected_shipments_columns))
missing_columns = get_unexpected_values(set(expected_shipments_columns),set(shipments_df.columns))

print('Expected columns are:',expected_shipments_columns)
if len(new_columns) > 0:
    print('These unexpected columns found:\n',new_columns)
elif len(missing_columns) > 0:
    print('These columns are missing:\n',missing_columns)
else:
    print('\nNo column discrepancies found.')

Expected columns are: ['State', 'Station Name (LEA)', 'Requisition ID', 'FSC', 'NIIN', 'Item Name', 'UI', 'Quantity', 'Acquisition Value', 'Date Shipped', 'Justification']

No column discrepancies found.


###### QUESTION: How many unique values are in each column of the 'SHIPMENTS' dataframe?

In [11]:
shipments_df.groupby('State').nunique()

,State,Station Name (LEA),Requisition ID,FSC,NIIN,Item Name,UI,Quantity,Acquisition Value,Date Shipped,Justification
State,,,,,,,,,,,
AK,1,1,32,8,17,12,4,9,14,5,10
AL,1,30,195,75,151,135,8,27,158,48,156
AR,1,13,101,40,69,67,6,14,74,23,70
AZ,1,14,63,39,56,55,6,16,51,22,62
CA,1,27,1083,139,812,495,16,93,691,48,778
CO,1,15,106,35,83,69,5,20,73,29,74
CT,1,7,20,12,18,18,2,8,18,12,18
DC,1,1,12,5,11,7,2,2,7,2,6
DE,1,2,100,19,86,32,3,15,71,5,49


###### QUESTION: How many null/NaN values are in the 'SHIPMENTS' dataframe?

In [12]:
shipments_df.isnull().sum()

State                 0
Station Name (LEA)    0
Requisition ID        0
FSC                   0
NIIN                  0
Item Name             0
UI                    0
Quantity              0
Acquisition Value     0
Date Shipped          0
Justification         0
dtype: int64

###### QUESTION: What can we learn about the 'Requistion ID' values in the 'SHIPMENTS' dataframe?

In [13]:
print('Length,Count:')
print(dict(shipments_df['Requisition ID'].str.len().value_counts()))
print('Number of unique values:',shipments_df['Requisition ID'].nunique())

Length,Count:
{14: 6353}
Number of unique values: 6353


### Check the Cancellations dataframe

###### QUESTION: Do the 'State' abbreviations in the 'CANCELLATIONS' dataframe match US postal abbreviations?

In [14]:
incorrect_state_abbr = [state_abbr for state_abbr in cancellations_df['State'] 
                         if state_abbr not in expected_state_abbreviations]
if len(incorrect_state_abbr) > 0:
    print('These states have inconsistant state/territory abbreviations:',incorrect_state_abbr)
else:
    print('No inconsistant state abbreviations were found.')

No inconsistant state abbreviations were found.


###### QUESTION: What columns are in the 'CANCELLATIONS' dataframe?

In [15]:
new_columns = get_unexpected_values(set(cancellations_df.columns), set(expected_cancellations_columns))
missing_columns = get_unexpected_values(set(expected_cancellations_columns),set(cancellations_df.columns))

print('Expected columns are:',expected_cancellations_columns)
if len(new_columns) > 0:
    print('These unexpected columns found:\n',new_columns)
elif len(missing_columns) > 0:
    print('These columns are missing:\n',missing_columns)
else:
    print('\nNo column discrepancies found.')

Expected columns are: ['Cancelled By', 'RTD Ref', 'State', 'Station Name (LEA)', 'FSC', 'NIIN', 'Item Name', 'UI', 'Quantity', 'Acquisition Value', 'Date Requested', 'Justification', 'Reason Cancelled']

No column discrepancies found.


###### QUESTION: How many unique values are in each column of the 'CANCELLATIONS' dataframe?

In [16]:
cancellations_df.groupby('State').nunique()

,Cancelled By,RTD Ref,State,Station Name (LEA),FSC,NIIN,Item Name,UI,Quantity,Acquisition Value,Date Requested,Justification,Reason Cancelled
State,,,,,,,,,,,,,
AK,1,7,1,1,6,7,7,3,6,6,6,7,6
AL,3,278,1,46,79,160,148,6,24,174,200,257,139
AR,3,101,1,23,39,61,59,6,18,65,66,80,51
AZ,3,65,1,12,35,52,47,4,19,53,42,54,36
CA,3,634,1,32,119,449,344,14,66,410,321,460,305
CO,3,78,1,17,31,61,55,4,17,58,58,72,46
CT,4,50,1,11,14,35,32,3,18,38,40,44,20
DC,2,29,1,2,13,23,19,4,21,22,10,12,12
DE,3,33,1,3,15,21,21,2,10,25,16,23,11


###### QUESTION: What unique values are found in the 'Cancelled By' column of the 'CANCELLATION dataframe?

In [17]:
cancellations_df['Cancelled By'].unique()

array(['LESOHQ', 'STATE', 'SYSTEM', 'UNKNOWN'], dtype=object)

###### QUESTION: How many null/NaN values are in the 'CANCELLATIONS' dataframe?

In [18]:
cancellations_df.isnull().sum()

Cancelled By            0
RTD Ref                 0
State                   0
Station Name (LEA)      0
FSC                     0
NIIN                    0
Item Name               0
UI                      0
Quantity                0
Acquisition Value       0
Date Requested          0
Justification           9
Reason Cancelled      214
dtype: int64

###### QUESTION: What can we learn about the 'RTD Ref' values in the 'CANCELLATIONS' dataframe?

In [19]:
print('Length,Count:')
print(dict(cancellations_df['RTD Ref'].astype(str).str.len().value_counts()))
print('Number of unique values:',cancellations_df['RTD Ref'].nunique())

Length,Count:
{7: 6635, 6: 5}
Number of unique values: 6640
